# Canonical Correlation Analysis (CCA)
Here I calculate the Canonical Correlation Coefficients and the canonical variables for the two datasets. 
I also plot the correlation coefficients and the canonical variables.

This analysis is based on the following article:
https://brainder.org/2019/12/27/redundancy-in-canonical-correlation-analysis/


In [1]:
# import libraries
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cross_decomposition import CCA
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
# Parameters
cell_type = "PBMC"
Shuffle = True

In [3]:
# set paths to data
morphology_data_path = pathlib.Path(
    f"../../data/{cell_type}_preprocessed_sc_norm_aggregated.parquet"
).resolve(strict=True)
nomic_data_path = pathlib.Path(
    f"../../2.Nomic_nELISA_Analysis/Data/clean/Plate2/nELISA_plate_430420_{cell_type}_clean.parquet"
).resolve(strict=True)

# output path
results_file_path = pathlib.Path(f"../results/{cell_type}_redundancy_analysis.csv")
results_file_path.parent.mkdir(parents=True, exist_ok=True)

# read data
morphology_data = pd.read_parquet(morphology_data_path)
nomic_data = pd.read_parquet(nomic_data_path)

In [4]:
# get the columns that contain metadata
morphology_metadata = morphology_data[
    morphology_data.columns[morphology_data.columns.str.contains("Metadata")]
]
morphology_data = morphology_data.drop(morphology_metadata.columns, axis=1)

nomic_data_values = nomic_data[
    nomic_data.columns[nomic_data.columns.str.contains("[NSU]", regex=True)]
]
nomic_metadata = nomic_data.drop(nomic_data_values.columns, axis=1)

#### Data Needs to be in standard scalar format for CCA

In [5]:
# standardize the data for nomic standard scalar
scaler = StandardScaler()
nomic_data_values = scaler.fit_transform(nomic_data_values)
nomic_data_values = pd.DataFrame(
    nomic_data_values,
    columns=nomic_data.columns[nomic_data.columns.str.contains("[NSU]", regex=True)],
)

In [6]:
# check the scale of the data
nomic_data_values.describe()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
count,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,154.000000,...,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,154.000000,1.540000e+02
mean,-1.038131e-16,2.883696e-17,3.784851e-16,-1.787892e-16,-3.388343e-16,5.767392e-18,-4.613914e-17,1.557196e-16,-5.306001e-16,0.000000,...,6.113436e-16,4.613914e-17,-2.306957e-17,2.306957e-17,2.883696e-18,-3.258577e-16,-2.306957e-16,6.920871e-17,0.000000,3.979501e-16
std,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263,...,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263e+00,1.003263,1.003263e+00
min,-7.753361e-01,-2.404660e+00,-1.626729e+00,-2.623379e+00,-2.502733e+00,-2.063749e+00,-3.267475e+00,-2.126498e+00,-2.744970e+00,-2.045397,...,-2.444083e+00,-2.359350e+00,-2.390540e+00,-2.812137e+00,-2.100477e+00,-3.241260e+00,-2.075041e+00,-1.405828e+00,-2.423406,-2.034209e+00
25%,-5.852376e-01,-7.977280e-01,-7.345144e-01,-6.888167e-01,-7.773053e-01,-6.836681e-01,-7.810839e-01,-6.254525e-01,-5.719509e-01,-0.664659,...,-7.489428e-01,-6.543184e-01,-7.170485e-01,-5.184526e-01,-6.810349e-01,-7.408295e-01,-6.358911e-01,-6.591589e-01,-0.681727,-6.995932e-01
50%,-5.077906e-01,-6.492239e-02,-2.339748e-01,-1.011396e-01,-8.989044e-02,2.056610e-02,9.340619e-02,-8.593829e-02,-1.425891e-01,-0.229429,...,-3.713431e-02,-8.319940e-02,1.192415e-02,-7.199159e-02,-7.061398e-02,7.170670e-02,-1.133556e-01,-3.474098e-01,0.033378,-2.057293e-01
75%,2.378032e-01,6.132403e-01,8.189925e-01,5.894068e-01,6.385160e-01,6.156226e-01,7.638540e-01,4.953480e-01,4.344105e-01,0.502222,...,6.029268e-01,6.905801e-01,6.410902e-01,5.976565e-01,5.186115e-01,7.108837e-01,5.324321e-01,4.257470e-01,0.697235,6.313630e-01
max,2.910307e+00,2.647989e+00,2.247566e+00,3.007529e+00,2.991157e+00,3.399755e+00,2.458591e+00,3.847185e+00,3.326857e+00,3.710686,...,2.714300e+00,3.273316e+00,3.549740e+00,3.463280e+00,2.442682e+00,2.662569e+00,2.864553e+00,3.071173e+00,2.991102,3.797019e+00


In [7]:
# shuffle the data both rows and columns
if Shuffle:
    for column in nomic_data_values:
        np.random.shuffle(nomic_data_values[column].values)
    for column in morphology_data:
        np.random.shuffle(morphology_data[column].values)

### Variables
$Y_{N \times P} = MorphologyData$  
$X_{N \times Q} = NomicData$  
Where  
$N = Rows of each data set$   
note that each data set is paired so N is the same for both  
$P = Columns of MorphologyData$  
$Q = Columns of NomicData$  
$K = Number of Canonical Variables$  
Where  
$K = min(P,Q)$  
unless $N < min(P,Q)$  
then $K = min(N, P, Q)$

In [8]:
# define the variables
N = morphology_data.shape[0]
P = morphology_data.shape[1]

N = nomic_data_values.shape[0]
Q = nomic_data_values.shape[1]
print("N:", N, "P:", P, "Q:", Q)
K = min(N, P, Q)
print("K:", K)

N: 154 P: 1245 Q: 187
K: 154


#### Calculate the Canonical Correlation Coefficients  
X = Morphology Data  
Y = Nomic Data

In [9]:
# define the cca model
cca = CCA(n_components=K)
# fit the model to the paired data sets
cca.fit(morphology_data, nomic_data_values)
# transform the data to the canonical space
# get the canonical coefficients for both data sets
X_c, Y_c = cca.transform(morphology_data, nomic_data_values)
# r2 score of the model fit
r2_model = [cca.score(morphology_data, nomic_data_values), X_c, Y_c][0]
print("The R2 score for the Canonical Correlation is:", r2_model)

The R2 score for the Canonical Correlation is: -5.375612848080362


/home/lippincm/miniconda3/envs/Interstellar/lib/python3.10/site-packages/sklearn/cross_decomposition/_pls.py:302: UserWarning: Y residual is constant at iteration 153
  warnings.warn(f"Y residual is constant at iteration {k}")


#### Extract the canonical loadings from the CCA
In the absence of scikit-learn canonical loadings.  
We would calculate the loads as follows:  
$\tilde{A} = corr(Y,U)$  
$\tilde{B} = corr(X,V)$  
Where $X$ and $Y$ are the original data matrices  
and $U$ and $V$ are the canonical variates

In [10]:
A_tilde = cca.x_loadings_.T
B_tilde = cca.y_loadings_.T

From the canonical coefficients we can calculate the variance extracted by each canonical variable.  
$u_k = \frac{1}{P} \sum^P_{p=1} \tilde a^2_{pk}$  
  
$v_k = \frac{1}{Q} \sum^Q_{q=1} \tilde b^2_{qk}$  
Where $k$ is the canonical variable number and $p$ and $q$ are the variables in the original data sets.


In [11]:
u_k = []
v_k = []
for i in A_tilde:
    u_k.append(np.mean(i**2))
for i in B_tilde:
    v_k.append(np.mean(i**2))

We can caluculate the r2 score for each canonical variable as follows:

In [12]:
# coefficients of determination for each canonical variable
r2 = r2_score(u_k, v_k)
r2

-22498.077451058954

We then caluclate the Redundancy Index (RI) for each canonical variable as follows:  
$RI_u = u_k * r^2_k$  
$RI_v = v_k * r^2_k$

In [13]:
# calculate the redundancy index for each canonical variable
RI_u = []
RI_v = []

for i in u_k:
    RI_u.append(i * r2)
for i in v_k:
    RI_v.append(i * r2)

We then caculate the total redundancy of both data sets as follows:  
#### $RI_{total} = \sum^K_{k=1} RI_u + \sum^K_{k=1} RI_v$  
From the total redundancy we can calculate the percentage contribution of each data set to the total redundancy as follows:  
#### $RI_{u\%} = \frac{\sum^K_{k=1} RI_u}{RI_{total}}$  
#### $RI_{v\%} = \frac{\sum^K_{k=1} RI_v}{RI_{total}}$

In [14]:
RI_u_min = np.min(RI_u)
RI_v_min = np.min(RI_v)
RI_u_max = np.max(RI_u)
RI_v_max = np.max(RI_v)
global_min = np.min([RI_u_min, RI_v_min])
global_max = np.max([RI_u_max, RI_v_max])

# Calulate the global redundancy index
global_RI_u_v = np.sum(RI_u) + np.sum(RI_v)
global_RI_u = np.sum(RI_u) / global_RI_u_v * 100
global_RI_v = np.sum(RI_v) / global_RI_u_v * 100

In [15]:
# make a dataframe of the results
results_df = pd.DataFrame(
    {
        "RI_u": RI_u,
        "RI_v": RI_v,
        "u_k": u_k,
        "v_k": v_k,
        "r2": r2,
        "Shuffle": Shuffle,
        "global_RI_u": global_RI_u,
        "global_RI_v": global_RI_v,
        "global_RI_u_v": global_RI_u_v,
        "global_min": global_min,
        "global_max": global_max,
    }
)
results_df.head(5)

,RI_u,RI_v,u_k,v_k,r2,Shuffle,global_RI_u,global_RI_v,global_RI_u_v,global_min,global_max
0,-20.680679,-269.941699,0.000919,0.011998,-22498.077451,True,7.541602,92.458398,-39019.982604,-928.382353,-0.0
1,-20.620108,-928.382353,0.000917,0.041265,-22498.077451,True,7.541602,92.458398,-39019.982604,-928.382353,-0.0
2,-20.331814,-202.780922,0.000904,0.009013,-22498.077451,True,7.541602,92.458398,-39019.982604,-928.382353,-0.0
3,-20.269692,-579.151852,0.000901,0.025742,-22498.077451,True,7.541602,92.458398,-39019.982604,-928.382353,-0.0
4,-20.081381,-215.744773,0.000893,0.009589,-22498.077451,True,7.541602,92.458398,-39019.982604,-928.382353,-0.0


In [16]:
# check for file existence
if results_file_path.is_file():
    print("The results file exists.")
    #  read the results file
    existing_file_df = pd.read_csv(results_file_path)
    # check for if it is full for shuffle type
    if len(existing_file_df["Shuffle"].unique()) > 1:
        # delete the existing file
        results_file_path.unlink()
    elif not existing_file_df["Shuffle"].unique() == Shuffle:
        pd.concat([existing_file_df, results_df]).to_csv(results_file_path, index=False)
else:
    results_df.to_csv(results_file_path, index=False)
    print("The results file is created.")

The results file exists.
